# Inference in Gaussian processes

* [mathematicalmonk on GP inference](https://youtu.be/UH1d2mxwet8)

We've seen that a GP is defined by a multivariate normal distribution over *outputs associated with some indices* or *states at some points in time*. Given a number of known states, we can get conditional distributions over one or more unobserved states. We can query the GP for certain indices/times.

Let's separate $Z$ into a known part $Z_b$ and unknown part $Za$:

$$
\begin{pmatrix}
    Z_a \\
    Z_b
\end{pmatrix}
\sim
\mathcal{N}(
    \begin{pmatrix}
        \mu_a \\
        \mu_b
    \end{pmatrix}
,
    \begin{pmatrix}
        K_{aa} && K_{ab} \\
        K_{ba} && K_{bb}
    \end{pmatrix}
)
$$

Given some concrete values $Z_b = z_b$, the conditional is another Gaussian:

$$
(Z_a | Z_b = z_b) \sim \mathcal{N}(m, D)
$$

with

$$
\begin{align}
m &= \mu_a + K_{ab} K_{bb}^{-1}(z_b - \mu_b) \\
D &= K_{aa} - K_{ab} K_{bb}^{-1} K_{ba}
\end{align}
$$


## Example: polynomials
Let's transform our linear kernel into a polynomial kernel. If you want to know more about which transforms are allowed for kernels, there's a [video](https://youtu.be/Sc5hOS5HqdY). In this case, we choose a polynomial with non-negative coefficients. Let's look at some samples for

$$
\begin{align}
\mu(i) &= 0 \\
k(i, j) &= 0.1 (ij)^3 + 0.3 (ij)^2 + ij
\end{align}
$$

In [ ]:
import numpy as np

from utilities import plot_gp
    
_ = plot_gp(
    mu=lambda i: 0,
    k=lambda i, j: np.polynomial.polynomial.polyval(i * j, [0.1, 0.3, 1, 0]),
    t=np.linspace(-5, 5, 8))

### Visualizing mean and variance
We can use $K$ to get a variance for each output over the whole process (the entries on its diagonal). Let's visualize that together with some more samples.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

from utilities import covariance, mean

mu = lambda i: 0
k = lambda i, j: np.polynomial.polynomial.polyval(i * j, [0.1, 0.3, 1, 0])
t = np.linspace(-5, 5, 50)
    
# These functions just evaluate mu and k given t
mu_ = mean(mu, t)
K = covariance(k, t, t)
# Get the range of two standard deviations for each output
two_sigma = 2 * np.sqrt(np.diag(K))
    
# Draw some samples
samples = np.random.multivariate_normal(mu_, K, 30)

# Plot
sns.set_style('darkgrid')
for s in samples:
    plt.plot(t, s, alpha=0.3)
plt.plot(t, mu_, label='μ', linewidth=3)
plt.fill_between(t, mu_ - two_sigma, mu_ + two_sigma, label='2σ', alpha=0.4)
plt.xlim(-5, 5)
plt.legend()
plt.show()

### Introducing observations
Let's say we've observed outputs $(-4, 10)$ and $(1, 3)$. Keeping the same $t$ as in the previous plot, we can calculate and visualize the conditional distribution.

In [ ]:
ta = t
mua = mu_
Kaa = K

tb = np.array([-4, 1])
Zb = np.array([10, 3])
mub = mean(mu, tb)
Kbb = covariance(k, tb, tb)

Kab = covariance(k, ta, tb)
Kbb_inv = np.linalg.inv(Kbb)

m = mua + Kab.dot(Kbb_inv).dot(Zb - mub)
# NOTE that we use Kab^T instead of implicitly computing Kba.
# They are equal because covariance matrices are always symmetric.
D = Kaa - Kab.dot(Kbb_inv).dot(Kab.T)
two_sigma_posterior = 2 * np.sqrt(np.diag(D))

# Plot
sns.set_style('darkgrid')
plt.figure(figsize=(12, 8))
plt.plot(ta, m, label='μ')
plt.fill_between(ta, m - two_sigma_posterior, m + two_sigma_posterior, label='2σ', alpha=0.4)
plt.scatter(tb, Zb, label='observations')
plt.xlim(-5, 5)
plt.legend()
plt.show()

## Intuition
In the plot above, you see how observations can be used to get a **posterior** GP. The posterior can only generate outputs that match the observations. Any sample from the posterior passes through the observed points (i.e. if the posterior index includes the observed index, the outputs match the observation at that index). Two GP characteristics are especially notable:

* Given a small set of observations and GP-hyperparameters ($\mu$ and $k$), we can compute a **posterior process for any indexes** we desire. Usually, we even assume that indexes come from $\mathbb{R}^n$. For the plot, we sample a set of real numbers in $[-5, 5]$ (selected by `np.linspace`). We could also just get a posterior distribution of the output at the single index $1000$.
* In addition to the mean or *expected process*, the posterior includes a measure of variance. You can see in the plot how the **variance is squeezed at the observed points** and how it expands when we're far away from observations.

As a final visualization, here are full posterior distributions for outputs at $2$ and $4$. You'll see that we're pretty sure the output is around $7.5$ in the first case, but not so sure about the second case.

In [ ]:
import scipy

from utilities import posterior

m_1, D_1 = posterior(mu, k, [2], tb, Zb)
m_2, D_2 = posterior(mu, k, [4], tb, Zb)

# Extract mean and standard deviation
# (item() extracts the scalar from an ndarray with only one element)
mean_1 = m_1.item()
std_dev_1 = np.sqrt(D_1.item())
mean_2 = m_2.item()
std_dev_2 = np.sqrt(D_2.item())

# Plot pdfs
Z = np.linspace(0, 40, 1000)
pdf_1 = scipy.stats.norm.pdf(Z, mean_1, std_dev_1)
pdf_2 = scipy.stats.norm.pdf(Z, mean_2, std_dev_2)
sns.set_style('darkgrid')
plt.plot(Z, pdf_1, label='PDF at t=2')
plt.plot(Z, pdf_2, label='PDF at t=4')
plt.legend()
plt.show()

## Noisy observations
We can add some uncertainty priors to our observations. The key step to this is, that we look at the true outputs $Z$ via some noisy proxy $Y = Z + \varepsilon$, introducing errors $\varepsilon \sim \mathcal{N}(0, \sigma^2I)$. Our additional new prior is the variance/uncertainty of observing outputs. The new GP is simply

$$
Y \sim \mathcal{N}(\mu, K + \sigma^2I)
$$

And the conditional step changes to

$$
(Y_a | Y_b = y_b) \sim \mathcal{N}(m, D)
$$

$$
\begin{align}
m &= \mu_a + K_{ab} (K_{bb} + \sigma^2 I)^{-1}(z_b - \mu_b) \\
D &= (K_{aa} + \sigma^2 I) - K_{ab} (K_{bb} + \sigma^2 I)^{-1} K_{ba}
\end{align}
$$

If you're familiar with numerical optimization, you might see how the uncertainty $\sigma^2$ acts like a regularizing term.

To conclude, here are some examples of our posterior with different uncertainties.

In [ ]:
import pandas as pd

from utilities import noisy_posterior

def get_noisy_posterior(sigma):
    m, D = noisy_posterior(mu, k, sigma, ta, tb, Zb)
    two_sigma = 2 * np.sqrt(np.diag(D))
    return np.column_stack((ta, m, two_sigma, np.repeat(sigma, len(ta))))

def plot_noisy_posterior(t, m, two_sigma, label, color):
    plt.plot(t, m, color=color)
    plt.fill_between(t, m - two_sigma, m + two_sigma, color=color, alpha=0.4)
    plt.scatter(tb, Zb, color=color)

posteriors = np.concatenate(
    [get_noisy_posterior(s) for s in (0.1, 0.2, 1, 5)],
    axis=0)

grid = sns.FacetGrid(
    pd.DataFrame(posteriors, columns=['t', 'm', 'two_sigma', 'sigma']),
    col='sigma',
    hue='sigma',
    col_wrap=2,
    size=6,
    aspect=1)

sns.set_style('ticks')
grid.map(plot_noisy_posterior, 't', 'm', 'two_sigma')
plt.show()